# Data Cleansing + Linear Regression with Airbnb

I will be using the SF Airbnb rental dataset from [Inside Airbnb](http://insideairbnb.com/get-the-data.html).

 - Impute missing values
 - Identify & substitute NaN values to the median of the column
 - Encode categorical features
 - Make a Linear regression Model
 - Calculate RMSE,MSE,MAE

In [0]:
%run "/Classroom-Setup"

Initialized classroom variables & functions...

Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

Declared various utility methods: 
 Declared untilStreamIsReady( name:String ) to control workflow 
 
 All done!

In [0]:
%fs ls "dbfs:/mnt/training/airbnb/sf-listings/sf-listings-2019-03-06.csv"

path,name,size
dbfs:/mnt/training/airbnb/sf-listings/sf-listings-2019-03-06.csv,sf-listings-2019-03-06.csv,34234636


Read the csv to see some lines

In [0]:
%fs head "dbfs:/mnt/training/airbnb/sf-listings/sf-listings-2019-03-06.csv"

[Truncated to first 65536 bytes]
id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
958,https://www.airbnb.com/rooms/958,20190306152813,2019-03-06,"Bright, Modern Garden Unit - 1BR/1B",New update: the house next door is under construction and there is the possibility of noise from 7am - 5pm. Our rates are discounted during this time period. Our bright garden unit overlooks a grassy backyard area with fruit trees and native plants. It is an oasis in a big city. The apartment comfortably fits a couple or small family. It is located on a cul de sac street that ends at lovely Duboce Park.,"Newly remodeled, modern, and bright garden unit in historic Victorian home. *New fixtures and finishes. *Organic cotton sheets and towels. *Zero VOC and non-toxic Yolo paint. *Organic and fair-trade teas, fresh local ground coffee. *Local art on walls. *Sofa bed and Queen bed are in the same room. More of a petite apartment with a separate room for dining and kitchen.","New update: the house next door is under construction and there is the possibility of noise from 7am - 5pm. Our rates are discounted during this time period. Our bright garden unit overlooks a grassy backyard area with fruit trees and native plants. It is an oasis in a big city. The apartment comfortably fits a couple or small family. It is located on a cul de sac street that ends at lovely Duboce Park. Newly remodeled, modern, and bright garden unit in historic Victorian home. *New fixtures and finishes. *Organic cotton sheets and towels. *Zero VOC and non-toxic Yolo paint. *Organic and fair-trade teas, fresh local ground coffee. *Local art on walls. *Sofa bed and Queen bed are in the same room. More of a petite apartment with a separate room for dining and kitchen. *Full access to patio and backyard (shared with us and our dog who live upstairs) *Beautiful garden with fruit trees, native plants and lawn *Washer and dryer *Children's toys *Charcoal grill A family of 4 lives upstairs ",none,"*Quiet cul de sac in friendly neighborhood *Steps away from grassy park with 2 playgrounds and Recreational Center *Very family-friendly neighborhood *Quaint shops, grocery stores and restaurants all within a 5-10 minute walk","Due to the fact that we have children and a dog, we are up early 7-8am and their footsteps or paws can be heard fr

Load the Airbnb dataset in.

In [0]:
import pandas as pd
filePath = "dbfs:/mnt/training/airbnb/sf-listings/sf-listings-2019-03-06.csv"

rawDF = spark.read.csv(filePath, header="true", inferSchema="true", multiLine="true", escape='"')


For the sake of simplicity, only keep certain columns from this dataset.

In [0]:
columnsToKeep = [
  "host_is_superhost",
  "cancellation_policy",
  "instant_bookable",
  "host_total_listings_count",
  "neighbourhood_cleansed",
  "latitude",
  "longitude",
  "property_type",
  "room_type",
  "accommodates",
  "bathrooms",
  "bedrooms",
  "beds",
  "bed_type",
  "minimum_nights",
  "number_of_reviews",
  "review_scores_rating",
  "review_scores_accuracy",
  "review_scores_cleanliness",
  "review_scores_checkin",
  "review_scores_communication",
  "review_scores_location",
  "review_scores_value",
  "price"]

baseDF = rawDF.toPandas()[columnsToKeep]

### Fixing Data Types

Change the type of the price column which has the substring ',' & '$'

In [0]:
fixedPriceDF=baseDF.copy()
#substitute the substring "$" and "," to "" using 'regex=True'
fixedPriceDF['price']=fixedPriceDF['price'].str.replace('$','',regex=True).str.replace(',','',regex=True).astype('float64')
fixedPriceDF

### Summary statistics

In [0]:
print(fixedPriceDF["price"].describe())

count     7151.000000
mean       213.654034
std        313.282220
min          0.000000
25%        100.000000
50%        150.000000
75%        235.000000
max      10000.000000
Name: price, dtype: float64


#Filter the "free" Airbnbs.

In [0]:
fixedPriceDF['price'].loc[fixedPriceDF["price"] == 0].count()

Out[10]: 1

Now only keep rows with a strictly positive *price*.

In [0]:
posPricesDF = fixedPriceDF.loc[fixedPriceDF["price"] > 0]

Look at the *min* and *max* values of the *minimum_nights* column:

In [0]:
posPricesDF[["minimum_nights"]].describe()

,minimum_nights
count,7.150000e+03
mean,1.400226e+04
std,1.182625e+06
min,1.000000e+00
25%,2.000000e+00
50%,4.000000e+00
75%,3.000000e+01
max,1.000000e+08


In [0]:
(posPricesDF[["minimum_nights",'price']].groupby("minimum_nights",as_index=False)
.count()
.rename(columns={'price':'count'})
.sort_values(["count","minimum_nights"],ascending=False))

,minimum_nights,count
19,30,2757
1,2,1455
0,1,1251
2,3,822
3,4,270
4,5,176
20,31,133
6,7,72
28,60,32
21,32,31


Filter out those records where the *minimum_nights* is greater then 365:

In [0]:
minNightsDF = posPricesDF[posPricesDF["minimum_nights"]<= 365]

minNightsDF[["minimum_nights"]].describe()

,minimum_nights
count,7146.000000
mean,15.817800
std,22.511624
min,1.000000
25%,2.000000
50%,4.000000
75%,30.000000
max,365.000000


### Nulls

There are a lot of different ways to handle null values. Sometimes, null can actually be a key indicator of the thing you are trying to predict (e.g. if you don't fill in certain portions of a form, probability of it getting approved decreases).

Some ways to handle nulls:
* Drop any records that contain nulls
* Numeric:
  * Replace them with mean/median/zero/etc.
* Categorical:
  * Replace them with the mode
  * Create a special category for null
* Use techniques like ALS which are designed to impute missing values

Select columns which have Nan values

In [0]:
#7150 is the max number of rows
columns_na=minNightsDF.loc[:,minNightsDF.isna().any()].columns.tolist()
columns_na

Out[15]: ['bathrooms',
 'bedrooms',
 'beds',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value']

In [0]:
#isna return true or false and sum convert to 0 & 1
minNightsDF.isna().sum()

Out[16]: host_is_superhost                 0
cancellation_policy               0
instant_bookable                  0
host_total_listings_count         0
neighbourhood_cleansed            0
latitude                          0
longitude                         0
property_type                     0
room_type                         0
accommodates                      0
bathrooms                        21
bedrooms                          2
beds                              7
bed_type                          0
minimum_nights                    0
number_of_reviews                 0
review_scores_rating           1421
review_scores_accuracy         1425
review_scores_cleanliness      1424
review_scores_checkin          1427
review_scores_communication    1423
review_scores_location         1427
review_scores_value            1428
price                             0
dtype: int64

Create a new column for each one which has NaN values and substitute the respective row with 1, otherwise set 0

In [0]:
import numpy as np
for x in columns_na:
  minNightsDF[x+'_na']=np.where(minNightsDF[x].isna(),1,0)
#show some row and cols with null values
view1=(minNightsDF[['bathrooms','bathrooms_na','bedrooms','bedrooms_na']]
       .loc[minNightsDF[['bathrooms','bathrooms_na','bedrooms','bedrooms_na']].isna().any(axis=1),:])
view1

<command-38970833875817>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minNightsDF[x+'_na']=np.where(minNightsDF[x].isna(),1,0)


bathrooms,bathrooms_na,bedrooms,bedrooms_na
null,1,1.0,0
null,1,1.0,0
null,1,1.0,0
null,1,1.0,0
null,1,1.0,0
null,1,1.0,0
null,1,1.0,0
null,1,1.0,0
null,1,1.0,0
null,1,1.0,0


### Transformers and Estimators

Every col should have 7146 rows

In [0]:
from sklearn.impute import SimpleImputer
#replace every nan value with the actual median of the column(only numeric cols)
Imputer=SimpleImputer(strategy='median')
numeric_cols=minNightsDF.select_dtypes(exclude='object').columns.tolist()
minNightsDF[numeric_cols]=Imputer.fit_transform(minNightsDF[numeric_cols])
minNightsDF.count()

<command-38970833875819>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minNightsDF[numeric_cols]=Imputer.fit_transform(minNightsDF[numeric_cols])
/databricks/python/lib/python3.8/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
Out[18]: host_is_superhost                 7146
cancellation_policy               7146
instant_bookable                  7146
host_total_listings_count         7146
neighbourhood_cl

I will save this DataFrame to Delta so that I can start building models with it.

In [0]:
outputPath = userhome + "/machine-learning-p/airbnb-cleansed.delta"

spark.createDataFrame(minNightsDF).write.format("delta").mode("overwrite").save(outputPath)

## Build a linear regression model to estimate the price of airbnb houses
 - Encode categorical features
 - Build a Linear regression Model
 - Calculate RMSE,MSE,MAE

In [0]:
airbnbDF=spark.read.format('delta').load(outputPath).toPandas()

In [0]:
df=airbnbDF.copy()

In [0]:
cols_numeric=[
'host_total_listings_count',
'latitude',
'longitude',
'accommodates',
'bathrooms',
'bedrooms',
'beds',
'minimum_nights',
'number_of_reviews',
'review_scores_rating',
'review_scores_accuracy',
'review_scores_cleanliness',
'review_scores_checkin',
'review_scores_communication',
'review_scores_location',
'review_scores_value']
cols_categorical=df.select_dtypes(include='object').columns.tolist()

# What Are the Basic Assumption?(favourite)
There are four assumptions associated with a linear regression model:

- Linearity: The relationship between X and the mean of Y is linear.
- Homoscedasticity: The variance of residual is the same for any value of X.
- Independence: Observations are independent of each other.
- Normality: For any fixed value of X, Y is normally distributed.

In [0]:
#standirzed the numerical columns
from sklearn.preprocessing import StandardScaler
SC=StandardScaler()
df[cols_numeric]=SC.fit_transform(df[cols_numeric])

In [0]:
#Use OHE in categorical features
import pandas as pd
df=pd.get_dummies(df,drop_first=True,columns=cols_categorical)
df.head(15)

,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,minimum_nights,number_of_reviews,review_scores_rating,...,property_type_Tiny house,property_type_Townhouse,property_type_Treehouse,property_type_Villa,room_type_Private room,room_type_Shared room,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed
0,-0.279596,-0.668999,-0.171527,0.417232,-0.413346,0.704651,0.200149,0.630039,-0.559106,0.630911,...,0,0,0,0,0,0,0,0,0,1
1,-0.285233,-0.890047,0.150528,1.461737,3.366268,2.848758,1.899808,-0.391727,-0.572893,0.630911,...,0,0,0,0,0,0,0,0,0,1
2,-0.290869,-0.558475,-0.199515,1.983990,2.106397,2.848758,1.899808,-0.480576,-0.476386,0.630911,...,0,0,0,0,0,0,0,0,0,1
3,-0.285233,-0.869185,-0.248402,-0.627273,-0.413346,-0.367403,-0.649681,0.630039,-0.462599,0.630911,...,0,0,0,0,0,0,0,0,0,1
4,-0.268324,0.950248,0.782694,0.417232,-0.413346,-0.367403,0.200149,-0.613850,-0.228224,0.312728,...,0,0,0,0,0,0,0,0,0,1
5,-0.285233,-0.150112,1.070416,0.939485,-0.413346,-0.367403,1.049978,0.630039,-0.490172,0.630911,...,0,0,0,0,1,0,0,0,0,1
6,-0.273960,-0.609076,0.167321,1.461737,0.846526,2.848758,1.899808,-0.525001,-0.324731,0.312728,...,0,0,0,0,0,0,0,0,0,1
7,-0.290869,-0.107944,-0.240565,-0.627273,-0.413346,-0.367403,-0.649681,0.630039,-0.517746,0.630911,...,0,0,0,0,0,0,0,0,0,1
8,-0.290869,-0.460379,-0.175632,0.417232,-0.413346,0.704651,0.200149,-0.569426,0.226739,0.312728,...,0,0,0,0,0,0,0,0,0,1
9,0.272777,1.313335,0.063950,-0.627273,-0.413346,-0.367403,-0.649681,0.630039,-0.572893,-5.732741,...,0,0,0,0,0,0,0,0,0,1


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

X=df.loc[:,df.columns!='price']
y=df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#Build the model
lr=LinearRegression()
lr.fit(X_train,y_train)
#Evaluate the model
y_pred=lr.predict(X_test)

print('The mse error is:',mean_squared_error(y_test,y_pred))
print('The rmse error is:',np.sqrt(mean_squared_error(y_test,y_pred)))
print('The mae error is:',mean_absolute_error(y_test,y_pred))
print('The value of R2 is:',r2_score(y_test,y_pred))

The mse error is: 40855.33974276679
The rmse error is: 202.12703862365072
The mae error is: 89.57785436460726
The value of R2 is: 0.2656381583289351


1. I will use the Lasso(L1) regression to compare values

In [0]:
from sklearn.linear_model import Lasso
#Build the model
la=Lasso(alpha=1)
la.fit(X_train,y_train)
#Evaluate the model
y_la_pred=lr.predict(X_test)
print('The mse error is:',mean_squared_error(y_test,y_la_pred))
print('The rmse error is:',np.sqrt(mean_squared_error(y_test,y_la_pred)))
print('The mae error is:',mean_absolute_error(y_test,y_la_pred))
print('The value of R2 is:',r2_score(y_test,y_la_pred))

The mse error is: 40855.33974276679
The rmse error is: 202.12703862365072
The mae error is: 89.57785436460726
The value of R2 is: 0.2656381583289351


1. I will use the Ridge(L2) regression to compare values

In [0]:
from sklearn.linear_model import Ridge
#Build the model
rd=Ridge(alpha=1)
rd.fit(X_train,y_train)
#Evaluate the model
y_rd_pred=rd.predict(X_test)
print('The mse error is:',mean_squared_error(y_test,y_rd_pred))
print('The rmse error is:',np.sqrt(mean_squared_error(y_test,y_rd_pred)))
print('The mae error is:',mean_absolute_error(y_test,y_rd_pred))
print('The value of R2 is:',r2_score(y_test,y_rd_pred))

The mse error is: 40842.30792952824
The rmse error is: 202.09479936289364
The mae error is: 89.48749837757455
The value of R2 is: 0.2658724010602508
